In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

In [2]:
import numpy as np
import os

In [3]:
from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [4]:
!wget https://github.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/raw/main/data/human-thermal-dataset.zip
!unzip -q human-thermal-dataset.zip

--2022-04-13 04:41:12--  https://github.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/raw/main/data/human-thermal-dataset.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/main/data/human-thermal-dataset.zip [following]
--2022-04-13 04:41:13--  https://raw.githubusercontent.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/main/data/human-thermal-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40972922 (39M) [application/zip]
Saving to: ‘human-thermal-dataset.zip’

human-thermal-datas 100%[====

In [4]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'human-thermal-dataset/train',
    'human-thermal-dataset/train',
    ['human']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'human-thermal-dataset/validate',
    'human-thermal-dataset/validate',
    ['human']
)

In [5]:
train_data.size

388

In [6]:
val_data.size

18

In [7]:
spec = model_spec.get('efficientdet_lite2')

In [8]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
24/24 [==============================] - 696s 27s/step - det_loss: 1.4789 - cls_loss: 0.9487 - box_loss: 0.0106 - reg_l2_loss: 0.0759 - loss: 1.5548 - learning_rate: 0.0140 - gradient_norm: 2.2444 - val_det_loss: 1.2379 - val_cls_loss: 0.6700 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0759 - val_loss: 1.3138
Epoch 2/20
24/24 [==============================] - 613s 26s/step - det_loss: 0.7436 - cls_loss: 0.4219 - box_loss: 0.0064 - reg_l2_loss: 0.0760 - loss: 0.8196 - learning_rate: 0.0197 - gradient_norm: 3.2427 - val_det_loss: 1.0340 - val_cls_loss: 0.6122 - val_box_loss: 0.0084 - val_reg_l2_loss: 0.0761 - val_loss: 1.1101
Epoch 3/20
24/24 [==============================] - 587s 24s/step - det_loss: 0.4960 - cls_loss: 0.2876 - box_loss: 0.0042 - reg_l2_loss: 0.0761 - loss: 0.5722 - learning_rate: 0.0191 - gradient_norm: 3.7082 - val_det_loss: 1.0509 - val_cls_loss: 0.7074 - val_box_loss: 0.0069 - val_reg_l2_loss: 0.0762 - val_loss: 1.1271
Epoch 4/20
24/24 [================

In [9]:
model.evaluate(val_data)

1/1 [==============================] - 12s 12s/step



{'AP': 0.7395625,
 'AP50': 0.9970297,
 'AP75': 0.8581274,
 'AP_/human': 0.7395625,
 'APl': 0.7602745,
 'APm': 0.35,
 'APs': -1.0,
 'ARl': 0.82222223,
 'ARm': 0.6,
 'ARmax1': 0.55789477,
 'ARmax10': 0.8105263,
 'ARmax100': 0.8105263,
 'ARs': -1.0}

In [10]:
model.export(export_dir='.', tflite_filename='human_thermal_efflite2.tflite')

In [11]:
model.evaluate_tflite('human_thermal_efflite2.tflite', val_data)

18/18 [==============================] - 168s 9s/step



{'AP': 0.7059496,
 'AP50': 1.0,
 'AP75': 0.83226556,
 'AP_/human': 0.7059496,
 'APl': 0.73400486,
 'APm': 0.25344828,
 'APs': -1.0,
 'ARl': 0.76666665,
 'ARm': 0.5,
 'ARmax1': 0.55789477,
 'ARmax10': 0.74210525,
 'ARmax100': 0.7526316,
 'ARs': -1.0}

In [12]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('human_thermal_efflite2.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>